In [1]:
import os

In [2]:
!pwd

/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Text_summarzer_project/research


In [3]:
os.chdir("../")

In [4]:
!pwd

/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Text_summarzer_project


In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [15]:
from textSummarizer.constants import *
from textSummarizer.utils.utils import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
            self,
            config_file_path=CONFIG_FILE_PATH,
            params_file_path=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [16]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.utils import get_size

In [17]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download with following info: \n{headers}")
        else:
            logger.info(
                f"""File {filename} of size: \
                    {get_size(Path(self.config.local_data_file))} \
                    already exists!"""
            )

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [18]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2024-02-06 21:29:28,718: INFO: utils: yaml file config/config.yaml loaded successfully:]
[2024-02-06 21:29:28,720: INFO: utils: yaml file params.yaml loaded successfully:]
[2024-02-06 21:29:28,722: INFO: utils: created directory at: artifacts:]
[2024-02-06 21:29:28,724: INFO: utils: created directory at: artifacts/data_ingestion:]
[2024-02-06 21:29:30,335: INFO: 247506385: artifacts/data_ingestion/data.zip download with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A304:1D673:6902B3:6C06DA:65C296A8
Accept-Ranges: bytes
Date: Tue, 06 Feb 2024 20:29:29 GMT
Via: 1.1 varnish
X-Served-By: cache-sof1510035-SOF
X-Cache: MISS